<a href="https://colab.research.google.com/github/Nuthan03/Tranfer_Learning-Feature-Extraction-/blob/main/Tansfer_learning_in_TF(Feature_extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tansfer Learning with TensorFlow (Feature Extraction)

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problems.
There are two main benefits:
1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own, then we can adapt those to our own data.